# Basics
このノートでは、pysparkを用いてデータフレームを基本的な操作する例を示した。

## SparkSessionを立てる
pythonからpysparkを使う時には、SparkSession.builder.getOrCreate()でセッションを立てる。
ローカルで実行する際にはこれでとりあえず使える。

## リソース管理
このままだと全てのCPU coreを使うので、制限したい場合はmaster("local[4]")のように指定する。
config("spark.driver.memory","2g")のようにセッションの設定値を指定する。

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
#spark = SparkSession.builder.master("local[4]") \
#                            .config("spark.driver.memory","2g") \
#                            .config("spark.executor.memory","2g") \
#                            .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/08 15:19:41 WARN Utils: Your hostname, TABLET-S9I8ER9S, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/08 15:19:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 15:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Parquetファイルを開く
- spark.read.parquet(): parquetファイルを開いてデータフレームを作成する。
- df: データフレーム
- df.show(): データフレームを表示。引数で行数指定。

In [2]:
df = spark.read.parquet("./data/clover_data.parquet")
df.show(10, truncate=False)

+-------------+---+-----+----+
|ts           |ch |cfd  |adc |
+-------------+---+-----+----+
|1657567439657|8  |23504|1348|
|1657568832534|6  |5435 |6750|
|1657569292316|2  |12850|744 |
|1657569301206|7  |14372|432 |
|1657569615821|11 |11872|900 |
|1657569880689|6  |31959|838 |
|1657569905859|6  |28153|344 |
|1657569962320|10 |4468 |552 |
|1657570040927|0  |31680|2531|
|1657570187406|2  |22049|456 |
+-------------+---+-----+----+
only showing top 10 rows


## Schemaの表示
printSchema()でデータフレームの構造を表示させることができる。

In [3]:
df.printSchema()

root
 |-- ts: long (nullable = true)
 |-- ch: short (nullable = true)
 |-- cfd: short (nullable = true)
 |-- adc: integer (nullable = true)



## その他関数
- count(): 行数をカウントする。

In [4]:
df.count()

858760

- select(): 指定した列だけ取り出す。

In [5]:
df = df.select("ch","adc","ts")
df.show(5)

+---+----+-------------+
| ch| adc|           ts|
+---+----+-------------+
|  8|1348|1657567439657|
|  6|6750|1657568832534|
|  2| 744|1657569292316|
|  7| 432|1657569301206|
| 11| 900|1657569615821|
+---+----+-------------+
only showing top 5 rows


- filter(): 条件を満たす行だけ取り出す。

In [6]:
df = df.filter("ch>=0 AND ch<4")
df.show(5)

+---+----+-------------+
| ch| adc|           ts|
+---+----+-------------+
|  2| 744|1657569292316|
|  0|2531|1657570040927|
|  2| 456|1657570187406|
|  0|   0|1657570307668|
|  3|3714|1657570429207|
+---+----+-------------+
only showing top 5 rows


## spark.sql.functions

様々なpyspark SQL関数が定義されている。以降、`F`としてインポートしておく
- F.col(): データフレームの列を返す
- F.lit(): 定数からなる列を返す
- withColumn(): 列を追加

In [7]:
from pyspark.sql import functions as F
# ts (ns)をsecに変換した列を定義
df.withColumn("ts_in_sec", F.lit(1.e-9)*F.col("ts")).show(5)

+---+----+-------------+------------------+
| ch| adc|           ts|         ts_in_sec|
+---+----+-------------+------------------+
|  2| 744|1657569292316|1657.5692923160002|
|  0|2531|1657570040927|    1657.570040927|
|  2| 456|1657570187406|    1657.570187406|
|  0|   0|1657570307668|1657.5703076680002|
|  3|3714|1657570429207|1657.5704292070002|
+---+----+-------------+------------------+
only showing top 5 rows


- F.expr(): データフレームの列に対してSQL表記の操作を行う。

上のセルのような操作を完結に表記できる。

In [8]:
df.withColumn("ts_in_sec", F.expr("1.e-9*ts")).show(5)

+---+----+-------------+------------------+
| ch| adc|           ts|         ts_in_sec|
+---+----+-------------+------------------+
|  2| 744|1657569292316|1657.5692923160002|
|  0|2531|1657570040927|    1657.570040927|
|  2| 456|1657570187406|    1657.570187406|
|  0|   0|1657570307668|1657.5703076680002|
|  3|3714|1657570429207|1657.5704292070002|
+---+----+-------------+------------------+
only showing top 5 rows


- spark.sql(): SQLコマンドを実行する。
- df.createOrReplaceTempView(): データフレームをSQLで呼ぶためのTempViewに登録する。

下はSQLで"ch"毎の"adc"の値の平均をとる操作を行うもの。

In [24]:
df.createOrReplaceTempView("df")
result_df = spark.sql("""
    SELECT ch, AVG(adc) AS avg_adc
    FROM df
    GROUP BY ch
""")
result_df.show()

+---+------------------+
| ch|           avg_adc|
+---+------------------+
|  1|1360.1958049005211|
|  3|3497.6671999375685|
|  2|1210.0397673898456|
|  0|3590.5568891736652|
+---+------------------+

